In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

from deepface.basemodels import VGGFace
tf_model = VGGFace.loadModel()

import torch
from core.face.vggface import VGGFace as PTVGGFace
torch_model = PTVGGFace()

weights_file = os.path.join(project_root, "pretrained/vggface.pth")

torch_model.load_weights(weights_file)

2023-06-13 11:13:55.785509: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 11:13:55.805712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 11:13:57.505707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20548 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2023-06-13 11:13:57.506031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device

VGGFace(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilati

AttributeError: 'VGGFace' object has no attribute 'load_weights'

In [15]:
import cv2
import numpy as np
test_img = '/home/jixian/tmp/face_0_cropped.jpg'
img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
tf_img = np.expand_dims(img, axis=0)

tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
# convert img to torch tensor

img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
img = img.astype(np.float32)
img = img.transpose(2, 0, 1)
torch_img = torch.from_numpy(img)

with torch.no_grad():
  torch_res = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
  print(torch_res[:10])

  torch_res2 = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
  print(torch_res2[:10])

[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[4.094322681427002, -1.0221033096313477, 2.4471354484558105, 1.0899137258529663, 2.423877239227295, 3.3153252601623535, 4.256931304931641, 2.577716588973999, 5.125513076782227, 3.544717788696289]
[1.3235158920288086, -0.8596339225769043, 1.1910004615783691, 0.5995866656303406, 3.1575140953063965, 0.6109924912452698, 4.83270788192749, 0.3029395341873169, 2.940319061279297, 1.6968272924423218]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([tf_res], [torch_res]))
print(cosine_similarity([tf_res], [torch_res2]))
print(cosine_similarity([torch_res], [torch_res2]))


[[0.99999999]]
[[0.99999999]]
[[1.]]
